In [ ]:
out_dir = "/workspace/nemo/vol/LBM_EB/preprocessed_data/LBM_EB"
in_dir = "/workspace/nemo/vol/LBM_EB/RawDataAligned/"

In [2]:
import yaml
from preprocessor.preprocessor_mp import Preprocessor
config = "/workspace/nemo/vol/dataStore2/FastSpeech2/config/LBM_EB/preprocess.yaml"
config = yaml.load(open(config, "r"), Loader=yaml.FullLoader)
preprocessor = Preprocessor(config)

/workspace/nemo/vol/dataStore2/FastSpeech2/audio/stft.py:42: FutureWarning: Pass size=1024 as keyword args. From version 0.10 passing these as positional arguments will result in an error
  fft_window = pad_center(fft_window, filter_length)
/workspace/nemo/vol/dataStore2/FastSpeech2/audio/stft.py:145: FutureWarning: Pass sr=22050, n_fft=1024, n_mels=80, fmin=0, fmax=8000 as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mel_basis = librosa_mel_fn(


In [3]:
preprocessor.in_dir = in_dir
preprocessor.out_dir = out_dir

In [4]:
# import os
# os.makedirs((os.path.join(preprocessor.out_dir, "pitch")), exist_ok=True)
# os.makedirs((os.path.join(preprocessor.out_dir, "energy")), exist_ok=True)
# os.makedirs((os.path.join(preprocessor.out_dir, "duration")), exist_ok=True)
# os.makedirs((os.path.join(preprocessor.out_dir, "mel")), exist_ok=True)
# os.makedirs((os.path.join(preprocessor.out_dir, "wav_modified")), exist_ok=True)

In [5]:
#preprocessor.collect_all_paths()

In [6]:
import pandas as pd
paths = pd.read_csv(preprocessor.out_dir + '/' + 'paths.csv')

In [7]:
paths.iloc[0]['tg_paths'].split('/')[-1].split('.')[0]

'legend_land_v1-13_various_64kb_2'

In [8]:
paths['emotions'] = paths['tg_paths'].apply(lambda item: item.split("/")[-2])
paths['speakers'] = paths['tg_paths'].apply(lambda item: item.split("/")[-3])

In [9]:
basenames = [i.split('.')[0].split('/')[-1] for i in paths['tg_paths'].to_list()]

In [10]:
emotions = paths['emotions'].to_list()
speakers = paths['speakers'].to_list()

In [11]:
len(basenames), len(emotions), len(speakers)

(1420214, 1420214, 1420214)

In [12]:
#preprocessor.process_duration_and_pitch_mp(basenames, speakers, emotions)

In [13]:
preprocessor.build_from_path(speakers, emotions, basenames)

906493it [22:33, 208.95it/s]  IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

1420214it [1:04:07, 369.09it/s]


Computing statistic quantities ...
Total time: 339.823778281683 hours


In [15]:
preprocessor.out_dir

'/workspace/nemo/vol/LBM_EB/preprocessed_data/LBM_EB'

In [17]:
import os
len(os.listdir(os.path.join(preprocessor.out_dir, "pitch")))

1317149

In [18]:
len(os.listdir(os.path.join(preprocessor.out_dir, "mel")))

1317149

In [19]:
len(os.listdir(os.path.join(preprocessor.out_dir, "energy")))

1317149

In [20]:
len(os.listdir(os.path.join(preprocessor.out_dir, "duration")))

1317149

In [23]:
len([i for i in os.walk(os.path.join(preprocessor.out_dir, "TextGrid"))])

20536

In [32]:
file_count = 0
tg_files = []
for speaker in os.listdir(os.path.join(preprocessor.out_dir, "TextGrid")):
    for emotion in os.listdir(os.path.join(preprocessor.out_dir, "TextGrid", speaker)):
        for file in os.listdir(os.path.join(preprocessor.out_dir, "TextGrid", speaker, emotion)):
            if file.split('.')[1] == 'TextGrid':
                tg_files.append(file)
                file_count += 1

In [27]:
file_count

1350853

In [28]:
with open(os.path.join(preprocessor.out_dir, 'train.txt'), 'r') as f:
    train_data = f.read()

In [35]:
with open(os.path.join(preprocessor.out_dir, 'val.txt'), 'r') as f:
    val_data = f.read()

In [36]:
len(train_data.split('\n')) + len(val_data.split('\n'))

1316890

In [31]:
file

'sistercarrie_41_dreiser_64kb_119.TextGrid'

In [41]:
#print(os.environ['RANK'])
#print(os.environ['WORLD_SIZE'])
#print(os.environ['MASTER_ADDR'])
#print(os.environ['MASTER_PORT'])

In [63]:
import json
with open('/workspace/nemo/vol/LBM_EB/preprocessed_data/LBM_EB/speakers.json', 'r') as f:
    speakers_dict = json.load(f)
    
with open('/workspace/nemo/vol/LBM_EB/preprocessed_data/LBM_EB/emotions.json', 'r') as f:
    emotions_dict = json.load(f)

In [64]:
emotions_dict

{'Undetected': 0,
 'Neutral': 1,
 'Anger': 2,
 'Hope': 3,
 'Confused': 4,
 'Sadness': 5,
 'Anticipation': 6,
 'Fear': 7,
 'Joy': 8,
 'Surprise': 9,
 'Disgust': 10,
 'Trust': 11,
 'Curious': 12}

In [68]:
batch_speakers = [ 431, 1432, 1432, 1878, 1359, 1573, 1171, 1740, 1454, 1589, 1615, 1489, 1709, 1653, 1573,  343]
batch_speakers_ = [key for value in batch_speakers for key, code in speakers_dict.items() if code == value]
batch_emotions = [ 1, 10,  5, 12, 11, 12, 12, 11,  5,  3, 12, 12, 12,  1, 12,  8]
batch_emotions_ = [key for value in batch_emotions for key, code in emotions_dict.items() if code == value]

batch_basenames = ['chunk_1', 'jesuits_19_parkman_64kb_7', 'pomonastravels_02_stockton_64kb_7',
                   'fairyland_of_science_10_buckley_64kb_95', 
                   '21stcenturypolicing_09_taskforce_64kb_13', 'westernunitedstates_06_fairbanks_64kb_21', 'chunk_60',
                   'wayofallflesh_12_butler_64kb_15', 'modernghoststories_05-2_scarborough_64kb_98',
                   'jackjill_23_alcott_64kb_24', 
                   'tombrowns_02_hughes_64kb_142', 'ayesha_return_11_haggard_64kb_25',
                   'allthingsconsidered_05_gkchesterton_64kb_105', 
                   'diary_of_uboat_commander_06_king-hall_64kb_64',
                   'westernunitedstates_15_fairbanks_64kb_22', 'story_my_life_06_keller_64kb_14']

In [71]:
for basename, speaker, emotion in zip(batch_basenames, batch_speakers_, batch_emotions_):
    basename = basename + '.TextGrid'
    text_filepath = os.path.join(preprocessor.out_dir, 'TextGrid', str(speaker), str(emotion), basename)
    with open(text_filepath, 'r') as f:
        d = f.read()

In [119]:
train_data_mod = []
val_data_mod = []
from tqdm import tqdm
for i, item in (enumerate(train_data.split('\n'))):
    text = item.split('|')[-1]
    if not len(text) > 2000:
        train_data_mod.append(item)
        
        
for i, item in (enumerate(val_data.split('\n'))):
    text = item.split('|')[-1]
    if not len(text) > 2000:
        val_data_mod.append(item)
#     basename, speaker, emotion = item.split('|')[:3] 
#     for basename_, speaker_, emotion_ in zip(batch_basenames, batch_speakers_, batch_emotions_) :
#         if basename == basename_ and speaker == speaker_ and emotion == emotion_:
#             items_.append(item)
#             continue

In [120]:
len(train_data_mod)

1216851

In [121]:
train_data_mod = train_data_mod[:-1]
val_data_mod = val_data_mod[:-1]

In [122]:
len(train_data_mod), len(val_data_mod)

(1216850, 99998)

In [123]:
train_data_mod = train_data_mod + val_data_mod[:90000]

In [124]:
len(train_data_mod)

1306850

In [125]:
val_data_mod = val_data_mod[90000:]

In [126]:
len(train_data_mod), len(val_data_mod)

(1306850, 9998)

In [128]:
train_data_len = len(train_data_mod)
val_data_len = len(val_data_mod)

with open(os.path.join(preprocessor.out_dir, "train.txt"), "w", encoding="utf-8") as f:
    for i, m in enumerate(train_data_mod):
        if i < train_data_len - 1:
            f.write(m + "\n")
        else:
            f.write(m)
        
with open(os.path.join(preprocessor.out_dir, "val.txt"), "w", encoding="utf-8") as f:
    for i, m in enumerate(val_data_mod):
        if i < val_data_len - 1:
            f.write(m + "\n")
        else:
            f.write(m)

In [98]:
val_data_mod[-1]

'flamingforest_23_curwood_64kb_46|2156|Curious|{HH IY1 R OW1 Z T UW1 HH IH0 Z F IY1 T AH0 N D P EY1 S T R EH1 S T L AH0 S L IY0 B AE1 K AH0 N D F AO1 R TH EH1 Z HH IY1 T R AY1 D T AH0 B R IH1 NG T AH0 G EH1 DH ER0 AH0 F Y UW1 T AE1 NG G AH0 L D B IH1 T S AH0 V DH AH0 P AH1 Z AH0 L spn HH IY1 HH ER1 D V OY1 S IH0 Z AW1 T S AY1 D spn AE1 N D V EH1 R IY0 S UW1 N F EH1 L T DH AH0 M UW1 V M AH0 N T AH0 V DH AH0 B AE0 T OW1 AH1 N D ER0 HH IH0 Z F IY1 T spn AE1 N D TH R UW1 W AH1 N AH0 V DH AH0 SH AO1 R W ER0 D W IH1 N D OW0 Z spn HH IY1 S AO1 T R IY1 Z AH0 N D S AE1 N D IY0 B IY1 CH spn S L OW1 L IY0 D R IH1 F T IH0 NG AH0 W EY1}|he rose to his feet and paced restlessly back and forth as he tried to bring together a few tangled bits of the puzzle. he heard voices outside, and very soon felt the movement of the bateaux under his feet, and through one of the shoreward windows he saw trees and sandy beach slowly drifting away. '

In [82]:
for item in items_:
    text = item.split('|')[-1]
    if len(text) > 2000:
        print(text)
    print(len(text))

242
245
271
271
274
265
315
253
windows secured setting device wall secured meds delivered throwing frank loading bolt concussion grenade w secured aiding injured friendly windows secured need to get straight loaded starting barricade clamor sit rice or wire deployed medical darted nets sea four deployed loading new magazine nitro deployed flash bang out concussion out securing door reload medical dart loaded door secured pows changing mags loading dark changing mags reloading loaded loading glass medical dart loaded loading dart friendly down gona get straight barricade sat deploying shield come on i'm just one day from retirement throwing smoke rented setting c four c four deployed smoke grenade throwing frog glass clamor deployed up too old for the shit changing mas loading new magg max loby west side maintenance area breaching i've got you stairs victoria subramania biao music room hangar bay supply room flush out charge set concussion charpy southern bar code device deployed floor

In [80]:
text

"you feel cold, saunders, said eustace, when you can't afford to support a great cat skin line coat like this. you do yourself very well, all things considered. look at those gloves, for instance who could possibly feel cold when wearing them? they are far too clumsy, though, for driving. "

In [101]:
import time

In [102]:
time.time()

1697114017.6951745